In [1]:
import os
import numpy as np
import pandas as pd
from skimage import io, color, exposure, filters
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans,BisectingKMeans,SpectralClustering,AgglomerativeClustering,DBSCAN
from sklearn.metrics import fowlkes_mallows_score,silhouette_score

import warnings 
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
crop_folder=r'Cropped'
classes=os.listdir(crop_folder)
def edge_histogram(images):
    imgs=[]
    classs=[]
    for idx,pt in enumerate(images):
        path = os.path.join(crop_folder,pt)
        for img in os.listdir(path):
            src_pt = os.path.join(path, img)
            image = io.imread(src_pt)
            gray_img = color.rgb2gray(image)
            dx, dy = filters.sobel_h(gray_img), filters.sobel_v(gray_img)
            angle_sobel = np.mod(np.arctan2(dy, dx), np.pi)
            hist, bins = exposure.histogram(angle_sobel, nbins=36)
            hist=hist/np.sum(hist)
            imgs.append(hist)
            classs.append(idx)
            
    imgs=np.array(imgs)
    classs=np.array(classs)
    return imgs,classs

imgs,classs=edge_histogram(classes)

In [ ]:
pca=PCA(n_components=2)
imgs=pca.fit_transform(imgs)

In [ ]:
#K-mean clustering and its variants for 𝐾 = 4: init=random

model= KMeans(init="random",n_clusters=4)
model.fit(imgs)
print("fowlkes_mallows_score :"+str(fowlkes_mallows_score(classs,model.labels_)))
print("silhouette_score :"+str(silhouette_score(imgs,model.labels_)))

In [ ]:
#init = kmeans++
model= KMeans(init="k-means++",n_clusters=4)
model.fit(imgs)
print("fowlkes_mallows_score :"+str(fowlkes_mallows_score(classs,model.labels_)))
print("silhouette_score :"+str(silhouette_score(imgs,model.labels_)))

In [ ]:
#Bisecting K-means
model= BisectingKMeans(init="random",n_clusters=4)
model.fit(imgs)
print("fowlkes_mallows_score :"+str(fowlkes_mallows_score(classs,model.labels_)))
print("silhouette_score :"+str(silhouette_score(imgs,model.labels_)))

In [ ]:
#spectral clustering
model= SpectralClustering(n_clusters=4)
model.fit(imgs)
print("fowlkes_mallows_score :"+str(fowlkes_mallows_score(classs,model.labels_)))
print("silhouette_score :"+str(silhouette_score(imgs,model.labels_)))

In [ ]:
#DBSCAN
model = DBSCAN(eps=0.0169, min_samples=2).fit(imgs)
labels = model.labels_
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
print("Estimated number of clusters: %d" % n_clusters_)
print("eps_used : %.4f" % model.eps)
print("min_samples : %d " % model.min_samples)
print("fowlkes_mallows_score :"+str(fowlkes_mallows_score(classs,labels)))
print("silhouette_score :"+str(silhouette_score(imgs,labels)))

In [ ]:
#Agglomerative clustering 
link=[]
sc=[]
fol=[]
for init in [ "single","complete", "average","ward"]:
    model = AgglomerativeClustering(n_clusters=4,linkage=init)
    predict=model.fit_predict(imgs)
    link.append(init)
    sc.append(silhouette_score(imgs,predict))
    fol.append(fowlkes_mallows_score(classs,predict))
score=pd.DataFrame({'link': link, 'silhouette': sc , 'fowlkes': fol})
score

##### Based on fowlkes score best to worst : agglomerative single ,Average,DBSCAN,complete,spectral,ward,k-means++,random,bisecting

##### Based on fowlkes score best to worst : agglomerative single ,Average,DBSCAN,ward,k-means++,random,bisecting,complete,spectral